## 作業

In [1]:
import requests
res = requests.get('https://news.ltn.com.tw/list/breakingnews')
res

<Response [200]>

In [3]:
#res.text

In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(res.text, 'lxml')

In [6]:
#soup

In [16]:
for li in soup.select('.list li'):
  #print(li)
  title = li.select_one('span.title').text
  dt = li.select_one('.time').text
  link = li.select_one('a.tit').get('href')
  print(title, dt, link)
  print('======================================')

剪接影片造假談話 CNN：拜登用假廣告攻擊川普 09:34 https://news.ltn.com.tw/news/world/breakingnews/3296734
9/19重要財經新聞一覽 09:32 https://ec.ltn.com.tw/article/breakingnews/3296759
米奇米妮萌助陣！這檔購物節超划算 3000現抵300特惠組最低46折 09:30 https://ent.ltn.com.tw/news/breakingnews/3296637
安倍參拜靖國神社 報告已卸下首相職務 09:29 https://news.ltn.com.tw/news/world/breakingnews/3296750
追思李登輝》蔡總統親頒褒揚令 追思文感念「把民主自由留給台灣」 09:28 https://news.ltn.com.tw/news/politics/breakingnews/3296678
追思李登輝》「三十九與九十八」 李坤儀撰文悼念祖父慈愛 09:27 https://news.ltn.com.tw/news/politics/breakingnews/3296736
NBA》詹皇僅拿16張第1名選票  湖人主帥仍力挺：他才是MVP 09:21 https://sports.ltn.com.tw/news/breakingnews/3296723
NBA》重提往事倡導精神健康 「愛神」自曝：好友救了我一命 09:20 https://sports.ltn.com.tw/news/breakingnews/3296707
意外捲入輕生事件 郭雪芙到處傻問「誰墜樓過」 09:20 https://ent.ltn.com.tw/news/breakingnews/3296654
MLB》洋基「園丁」發願再戰2021 美球迷無情狠酸：拜託不要 09:17 https://sports.ltn.com.tw/news/breakingnews/3296695
科技股下挫牽動美各大指數收黑 道瓊跌244點 09:16 https://ec.ltn.com.tw/article/breakingnews/3296681
追思李登輝》兩組民眾疑追思會場周邊抗議 警方帶至陳抗區 09:15 https://news.ltn.com.t

In [17]:
newsary = []
for li in soup.select('.list li'):
  title = li.select_one('span.title').text
  dt = li.select_one('.time').text
  link = li.select_one('a.tit').get('href')
  newsary.append({'title':title, 'dt':dt, 'link':link})
  

In [18]:
newsary[0]

{'dt': '09:34',
 'link': 'https://news.ltn.com.tw/news/world/breakingnews/3296734',
 'title': '剪接影片造假談話 CNN：拜登用假廣告攻擊川普'}

In [19]:
import pandas
newsdf = pandas.DataFrame(newsary)

In [21]:
newsdf.head()

,title,dt,link
0,剪接影片造假談話 CNN：拜登用假廣告攻擊川普,09:34,https://news.ltn.com.tw/news/world/breakingnew...
1,9/19重要財經新聞一覽,09:32,https://ec.ltn.com.tw/article/breakingnews/329...
2,米奇米妮萌助陣！這檔購物節超划算 3000現抵300特惠組最低46折,09:30,https://ent.ltn.com.tw/news/breakingnews/3296637
3,安倍參拜靖國神社 報告已卸下首相職務,09:29,https://news.ltn.com.tw/news/world/breakingnew...
4,追思李登輝》蔡總統親頒褒揚令 追思文感念「把民主自由留給台灣」,09:28,https://news.ltn.com.tw/news/politics/breaking...


In [22]:
newsdf.to_excel('ltnnews.xlsx')

## 完整自由時報爬蟲

In [25]:
import requests
res = requests.get('https://news.ltn.com.tw/ajax/breakingnews/all/2')
jd = res.json()

In [28]:
newsdf = pandas.DataFrame(jd['data'].values())

In [30]:
newsdf.head(3)

,no,title,bigphoto_flag,photo_S,photo_L,url,time,type_en,group,type_cn,local,summary,video,localUrl,tagUrl,style,tagText
0,3296731,追思李登輝》李坤儀手捧骨灰罈 移靈車隊由翠山莊出發,0,http://img.ltn.com.tw/Upload/news/250/2020/09/...,http://img.ltn.com.tw/Upload/news/600/2020/09/...,https://news.ltn.com.tw/news/politics/breaking...,09:13,politics,breakingnews,政治,臺北市,前總統李登輝追思告別禮拜今天在淡水真理大學大禮拜堂舉行，移靈車隊上午約8時45分從翠山莊出發...,0,list/breakingnews/Taipei,list/breakingnews/politics,None,政治
1,3296724,營區停車場猥褻女同袍：「妹妹，你可以幫我嗎？」,0,http://img.ltn.com.tw/Upload/news/250/2020/09/...,http://img.ltn.com.tw/Upload/news/600/2020/09/...,https://news.ltn.com.tw/news/society/breakingn...,09:10,society,breakingnews,社會,高雄市,高市江姓職業軍人藉故約女同袍小真（化名）到營區停車場，在自小客車內涉嫌強行親吻、揉胸，甚至抓...,0,list/breakingnews/Kaohsiung,list/breakingnews/society,None,社會
2,3296649,「雙野CP」默契十足 綾野剛狂嗑烏龍麵直到極限,0,http://img.ltn.com.tw/Upload/ent/page/400S/202...,http://img.ltn.com.tw/Upload/ent/page/800S/202...,https://ent.ltn.com.tw/news/breakingnews/3296649,09:10,entertainment,breakingnews,娛樂,False,星野源、綾野剛主演TBS台日劇《MIU404》日前播畢，全劇收視率創下每集均超過雙位數的亮眼...,0,,list/breakingnews/entertainment,None,娛樂


In [33]:
import requests
import time
newsdfs = []
for i in range(2,11):
  res = requests.get('https://news.ltn.com.tw/ajax/breakingnews/all/{}'.format(i))
  jd = res.json()
  newsdf = pandas.DataFrame(jd['data'].values())
  time.sleep(1)
  newsdfs.append(newsdf)

In [35]:
type(newsdfs)

list

In [37]:
type(newsdfs[0])

pandas.core.frame.DataFrame

In [39]:
res = pandas.concat(newsdfs)

In [43]:
import jieba
words = []
for title in res['title'].tolist():
  words = words + jieba.lcut(title)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.915 seconds.
Prefix dict has been built successfully.


In [46]:
from collections import Counter
words = [word for word in words if len(word) >=2]
c = Counter(words)
c.most_common(10)

[('中職', 12),
 ('追思', 7),
 ('12', 7),
 ('中國', 7),
 ('李登輝', 6),
 ('影音', 6),
 ('兄弟', 6),
 ('網球', 5),
 ('NBA', 5),
 ('回應', 5)]